# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from sklearn.feature_extraction.text import (
    CountVectorizer
)
from sklearn.feature_selection import (
    SelectKBest, SelectPercentile,
    SelectFpr, VarianceThreshold,
    SelectFromModel,
    chi2, f_classif,
)
from sklearn.ensemble import (
    RandomForestClassifier
)
from sklearn.linear_model import (
    LogisticRegression
)
from sklearn.pipeline import (
    FeatureUnion, Pipeline
)
from scipy.sparse import hstack


%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
display(
    X_train.isnull().sum()[X_train.isnull().sum() > 0],
    X_test.isnull().sum()[X_test.isnull().sum() > 0]
)

funder                3635
installer             3655
subvillage             371
public_meeting        3334
scheme_management     3877
scheme_name          28166
permit                3056
dtype: int64

funder                869
installer             877
subvillage             99
public_meeting        821
scheme_management     969
scheme_name          7092
permit                737
dtype: int64

In [5]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [6]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [7]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

X_train.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_train.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))

X_train.loc[:, 'date_recorded_month'] /= 12
X_test.loc[:, 'date_recorded_month'] /= 12

In [8]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [9]:
X_train.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)
X_test.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)

In [10]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population']).fillna(0)
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population']).fillna(0)

In [11]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [12]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [13]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [14]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [15]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [16]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [17]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [18]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [19]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [20]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [21]:
drop_cats = ['source_type', 'source_class',
              'water_quality', 'quality_group',
              'quantity_group', 'waterpoint_type_group',
              'payment_type', 'extraction_type_group',
              'extraction_type_class', 'management_group'
              ]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [22]:
X_train.iloc[0,:]

amount_tsh                                              6000
funder                                                 Roman
gps_height                                              1390
installer                                              Roman
longitude                                            34.9381
latitude                                            -9.85632
wpt_name                                                   1
basin                                             Lake Nyasa
subvillage                                          Mnyusi B
region                                                Iringa
region_code                                               11
district_code                                              5
lga                                                   Ludewa
ward                                                Mundindi
population                                               109
public_meeting                                             1
scheme_management       

In [23]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []
text_selfeat_names = []

In [24]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [25]:
# lonlat shall be clusterized before usage
# day probably doesn't matter, but month might be (seasonality)
# the years in test and train do not intersect
X_train.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')
X_test.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')

In [26]:
sel = Pipeline([
    ('varthres', VarianceThreshold()),
    ('chi2', SelectFpr(chi2, 1e-19)),
    #('interact', PolynomialFeatures(2, interaction_only=True)),
    #('feats', FeatureUnion([
        # ('anova', SelectFpr(f_classif, 1e-19)), # yeah, i'm gonna be very pretentious!
        # ('chi2', SelectKBest(chi2, min(128, tr.shape[1])).fit(tr, ytr),)
        # OHE-encoded features are supposed to have additive impact, so linear model:
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=40)))
    #])),
])
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = CountVectorizer(ngram_range=(1,2), binary=True)
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    #kbest = SelectKBest(chi2, min(8, tr.shape[1])).fit(tr, ytr)

    sel.fit(tr, ytr)
    tr = sel.transform(tr)
    te = sel.transform(te)
    for i in range(tr.shape[1]):
        text_selfeat_names.append('%s_%s' % (cat, i))
    #tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    #te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    if tr.shape[1] > 0:
        text_feats_train.append(tr)
        text_feats_test.append(te)

text_feats_train = hstack(text_feats_train) # pd.DataFrame(hstack(text_feats_train), columns=text_selfeat_names)
text_feats_test = hstack(text_feats_test) # pd.DataFrame(hstack(text_feats_test), columns=text_selfeat_names)
# print(sel.fit_transform(text_feats_train, ytr).toarray().shape)
# sel_text_feats = SelectKBest(chi2, 48).fit(text_feats_train, ytr) # SelectFpr(f_classif, 1e-19)
sel_text_feats = sel
text_feats_train = sel.fit_transform(text_feats_train, ytr)
text_feats_test = sel.transform(text_feats_test)
print('cat features shape: ', text_feats_train.shape)
# print('top catfeats: ', sel.named_steps['linear'].estimator_.coef_[:20] )
del tr, te
X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore'),
                     pd.DataFrame(text_feats_train.toarray())],
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore'),
                    pd.DataFrame(text_feats_test.toarray())],
                    axis=1)

cat features shape:  (59400, 39)


In [27]:
X_train.head()

,amount_tsh,gps_height,wpt_name,population,public_meeting,permit,construction_year,date_recorded_month,dry_season,rainy_season,...,29,30,31,32,33,34,35,36,37,38
0,6000.0,1390,1,109,1.0,0.0,1999,0.25,1,0,...,0,1,0,1,1,0,0,0,1,0
1,0.0,1399,1,280,0.5,1.0,2010,0.25,1,0,...,0,0,0,0,0,0,0,0,1,0
2,25.0,686,1,250,1.0,1.0,2009,0.166666666667,1,0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,263,1,58,1.0,1.0,1986,0.0833333333333,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0,1,0,1.0,1.0,0,0.583333333333,0,0,...,0,0,0,0,0,0,0,0,1,0


In [28]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('wpt_name', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('construction_year', dtype('O')),
 ('date_recorded_month', dtype('O')),
 ('dry_season', dtype('O')),
 ('rainy_season', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [29]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
#y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
#display(
#    np.where(ytr != 0)[0][:40],
#    np.where(ycv != 0)[0][:40],
#    np.where(yva != 0)[0][:40],)
#ytr = y_bin.transform(ytr)
#ycv = y_bin.transform(ycv)
#yva = y_bin.transform(yva)

In [30]:
def y_to_df(y, ids):
    #y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [31]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'wpt_name',
 'population',
 'public_meeting',
 'permit',
 'construction_year',
 'date_recorded_month',
 'dry_season',
 'rainy_season',
 'age',
 'amount_per_man',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38]

In [32]:
X_train.to_csv('X_train_prepr2.csv')
X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
pd.Series(ytr).to_csv('y_train_prepr2.csv')
pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [33]:
import catboost

In [ ]:
clf = catboost.CatBoostClassifier(1800, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importances=True)

In [ ]:
clf.fit(xtr, ytr, cat_features=[xtr.columns.get_loc(c) for c in text_features if c in xtr.columns],
        eval_set=(xcv, ycv), verbose=True)

Borders generated
0:	learn 0.7029198134	test 0.700493938	bestTest 0.700493938	passed: 0.322 sec	total: 507ms	remaining: 15m 12s
1:	learn 0.7011640772	test 0.7000449035	bestTest 0.700493938	passed: 0.315 sec	total: 823ms	remaining: 12m 19s
2:	learn 0.7009235653	test 0.6999326448	bestTest 0.700493938	passed: 0.261 sec	total: 1.08s	remaining: 10m 48s
3:	learn 0.7000096205	test 0.7009429726	bestTest 0.7009429726	passed: 0.243 sec	total: 1.33s	remaining: 9m 55s
4:	learn 0.7002260811	test 0.7009429726	bestTest 0.7009429726	passed: 0.278 sec	total: 1.6s	remaining: 9m 36s
5:	learn 0.7084756362	test 0.7081275258	bestTest 0.7081275258	passed: 0.293 sec	total: 1.9s	remaining: 9m 27s
6:	learn 0.7043147819	test 0.7033004041	bestTest 0.7081275258	passed: 0.237 sec	total: 2.14s	remaining: 9m 7s
7:	learn 0.7003944394	test 0.7000449035	bestTest 0.7081275258	passed: 0.249 sec	total: 2.38s	remaining: 8m 54s
8:	learn 0.7055894944	test 0.703973956	bestTest 0.7081275258	passed: 0.245 sec	total: 2.63s	remain

74:	learn 0.721222762	test 0.7181185451	bestTest 0.7181185451	passed: 0.367 sec	total: 21.3s	remaining: 8m 8s
75:	learn 0.7216316321	test 0.7181185451	bestTest 0.7181185451	passed: 0.352 sec	total: 21.6s	remaining: 8m 10s
76:	learn 0.7217999904	test 0.718792097	bestTest 0.718792097	passed: 0.343 sec	total: 22s	remaining: 8m 11s
77:	learn 0.7226658329	test 0.7195779075	bestTest 0.7195779075	passed: 0.315 sec	total: 22.3s	remaining: 8m 11s
78:	learn 0.7225215258	test 0.7195779075	bestTest 0.7195779075	passed: 0.246 sec	total: 22.5s	remaining: 8m 10s
79:	learn 0.7223531675	test 0.7196901661	bestTest 0.7196901661	passed: 0.244 sec	total: 22.8s	remaining: 8m 9s
80:	learn 0.7235076242	test 0.7199146834	bestTest 0.7199146834	passed: 0.277 sec	total: 23s	remaining: 8m 8s
81:	learn 0.7238202896	test 0.7209250112	bestTest 0.7209250112	passed: 0.263 sec	total: 23.3s	remaining: 8m 8s
82:	learn 0.7233633171	test 0.7202514594	bestTest 0.7209250112	passed: 0.265 sec	total: 23.6s	remaining: 8m 7s
83:	

148:	learn 0.7357496753	test 0.7331612034	bestTest 0.7331612034	passed: 0.252 sec	total: 43.2s	remaining: 7m 58s
149:	learn 0.7358218289	test 0.7331612034	bestTest 0.7331612034	passed: 0.34 sec	total: 43.5s	remaining: 7m 58s
150:	learn 0.7360382895	test 0.7331612034	bestTest 0.7331612034	passed: 0.307 sec	total: 43.8s	remaining: 7m 58s
151:	learn 0.7359901871	test 0.7332734621	bestTest 0.7332734621	passed: 0.237 sec	total: 44s	remaining: 7m 57s
152:	learn 0.736375006	test 0.7337224966	bestTest 0.7337224966	passed: 0.244 sec	total: 44.3s	remaining: 7m 56s
153:	learn 0.7369281832	test 0.7341715312	bestTest 0.7341715312	passed: 0.254 sec	total: 44.5s	remaining: 7m 56s
154:	learn 0.7374092068	test 0.7342837899	bestTest 0.7342837899	passed: 0.298 sec	total: 44.8s	remaining: 7m 55s
155:	learn 0.7372167973	test 0.7340592726	bestTest 0.7342837899	passed: 0.303 sec	total: 45.1s	remaining: 7m 55s
156:	learn 0.7379383328	test 0.7349573417	bestTest 0.7349573417	passed: 0.261 sec	total: 45.4s	remai

221:	learn 0.7470296792	test 0.7404580153	bestTest 0.7404580153	passed: 0.249 sec	total: 1m 5s	remaining: 7m 42s
222:	learn 0.7469815768	test 0.7401212393	bestTest 0.7404580153	passed: 0.235 sec	total: 1m 5s	remaining: 7m 42s
223:	learn 0.7471499351	test 0.7403457566	bestTest 0.7404580153	passed: 0.249 sec	total: 1m 5s	remaining: 7m 41s
224:	learn 0.7476309587	test 0.7403457566	bestTest 0.7404580153	passed: 0.327 sec	total: 1m 5s	remaining: 7m 41s
225:	learn 0.7479676752	test 0.7403457566	bestTest 0.7404580153	passed: 0.288 sec	total: 1m 6s	remaining: 7m 41s
226:	learn 0.7478233681	test 0.740233498	bestTest 0.7404580153	passed: 0.242 sec	total: 1m 6s	remaining: 7m 40s
227:	learn 0.747534754	test 0.7400089807	bestTest 0.7404580153	passed: 0.253 sec	total: 1m 6s	remaining: 7m 39s
228:	learn 0.7476069075	test 0.7403457566	bestTest 0.7404580153	passed: 0.245 sec	total: 1m 6s	remaining: 7m 39s
229:	learn 0.7476069075	test 0.740233498	bestTest 0.7404580153	passed: 0.268 sec	total: 1m 7s	rema

294:	learn 0.7549184665	test 0.746856758	bestTest 0.746856758	passed: 0.24 sec	total: 1m 26s	remaining: 7m 20s
295:	learn 0.754846313	test 0.7469690166	bestTest 0.7469690166	passed: 0.305 sec	total: 1m 26s	remaining: 7m 20s
296:	learn 0.7547982106	test 0.7471935339	bestTest 0.7471935339	passed: 0.259 sec	total: 1m 26s	remaining: 7m 19s
297:	learn 0.7550146712	test 0.7471935339	bestTest 0.7471935339	passed: 0.293 sec	total: 1m 27s	remaining: 7m 19s
298:	learn 0.7548703641	test 0.746856758	bestTest 0.7471935339	passed: 0.357 sec	total: 1m 27s	remaining: 7m 19s
299:	learn 0.7550146712	test 0.746856758	bestTest 0.7471935339	passed: 0.372 sec	total: 1m 27s	remaining: 7m 19s
300:	learn 0.7552311318	test 0.7473057925	bestTest 0.7473057925	passed: 0.292 sec	total: 1m 28s	remaining: 7m 19s
301:	learn 0.7552070807	test 0.7471935339	bestTest 0.7473057925	passed: 0.373 sec	total: 1m 28s	remaining: 7m 19s
302:	learn 0.7550627736	test 0.7469690166	bestTest 0.7473057925	passed: 0.383 sec	total: 1m 28

367:	learn 0.7586704507	test 0.7504490346	bestTest 0.7510103278	passed: 0.255 sec	total: 1m 47s	remaining: 6m 59s
368:	learn 0.7586463995	test 0.7507858105	bestTest 0.7510103278	passed: 0.241 sec	total: 1m 48s	remaining: 6m 59s
369:	learn 0.7588869113	test 0.7508980692	bestTest 0.7510103278	passed: 0.241 sec	total: 1m 48s	remaining: 6m 58s
370:	learn 0.7590552696	test 0.7511225864	bestTest 0.7511225864	passed: 0.39 sec	total: 1m 48s	remaining: 6m 58s
371:	learn 0.7590552696	test 0.7508980692	bestTest 0.7511225864	passed: 0.283 sec	total: 1m 49s	remaining: 6m 58s
372:	learn 0.7590312184	test 0.7508980692	bestTest 0.7511225864	passed: 0.246 sec	total: 1m 49s	remaining: 6m 57s
373:	learn 0.7589350137	test 0.7504490346	bestTest 0.7511225864	passed: 0.368 sec	total: 1m 49s	remaining: 6m 57s
374:	learn 0.7590793208	test 0.7506735519	bestTest 0.7511225864	passed: 0.281 sec	total: 1m 49s	remaining: 6m 57s
375:	learn 0.759103372	test 0.7504490346	bestTest 0.7511225864	passed: 0.249 sec	total: 1

440:	learn 0.7624705373	test 0.7535922766	bestTest 0.7535922766	passed: 0.317 sec	total: 2m 10s	remaining: 6m 42s
441:	learn 0.7625907932	test 0.7535922766	bestTest 0.7535922766	passed: 0.335 sec	total: 2m 10s	remaining: 6m 41s
442:	learn 0.7628553562	test 0.7535922766	bestTest 0.7535922766	passed: 0.284 sec	total: 2m 11s	remaining: 6m 41s
443:	learn 0.7628553562	test 0.7535922766	bestTest 0.7535922766	passed: 0.337 sec	total: 2m 11s	remaining: 6m 41s
444:	learn 0.7628072538	test 0.7535922766	bestTest 0.7535922766	passed: 0.33 sec	total: 2m 11s	remaining: 6m 41s
445:	learn 0.7627351003	test 0.7537045352	bestTest 0.7537045352	passed: 0.302 sec	total: 2m 12s	remaining: 6m 40s
446:	learn 0.7628553562	test 0.753480018	bestTest 0.7537045352	passed: 0.35 sec	total: 2m 12s	remaining: 6m 40s
447:	learn 0.7628794074	test 0.753480018	bestTest 0.7537045352	passed: 0.315 sec	total: 2m 12s	remaining: 6m 40s
448:	learn 0.7627591515	test 0.7535922766	bestTest 0.7537045352	passed: 0.33 sec	total: 2m 1

513:	learn 0.7657655491	test 0.7568477773	bestTest 0.7576335878	passed: 0.263 sec	total: 2m 30s	remaining: 6m 17s
514:	learn 0.7657896003	test 0.7568477773	bestTest 0.7576335878	passed: 0.287 sec	total: 2m 31s	remaining: 6m 16s
515:	learn 0.7658377026	test 0.7568477773	bestTest 0.7576335878	passed: 0.304 sec	total: 2m 31s	remaining: 6m 16s
516:	learn 0.765885805	test 0.7569600359	bestTest 0.7576335878	passed: 0.248 sec	total: 2m 31s	remaining: 6m 16s
517:	

In [ ]:
clf.score(xcv, ycv)

In [ ]:
clf.score(xva, yva)